Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [157]:
NAME = ""
COLLABORATORS = ""

---

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [158]:
NAME = ""
COLLABORATORS = ""

---

In [159]:
!pip install geojson
!pip install shapely
!pip install PyShp
!pip install networkx
!pip install osmnx
!pip install pandas

In [160]:
import geojson
import pandas as pd
import numpy as np
import networkx as nx
import time
import csv
import ast
import shapefile as shp
from shapely.geometry import Polygon,shape,MultiPolygon
import shapely.ops
import warnings
import math
from collections import deque
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

### Helper functions

In [161]:
def isDistrictContiguous(district_num, assignment, contiguity_list, print_isolates=False, ignore_list=[]):
    ## input:
    ## district_num: the district number
    ## assignment: the assignment from precinct to district
    ## contiguity_list: the list of neighbors for each precinct, from the csv file
    contiguity_list.columns = ['Precinct','Neighbors']
    district_graph = nx.Graph() #creates an empty undirected graph
    district_nodes = assignment[assignment['District']==district_num]['GEOID20'].tolist()
    for i in ignore_list:
        try:
            district_nodes.remove(i)
        except ValueError:
            pass
    district_graph.add_nodes_from(district_nodes)
    for id in district_nodes:
        neighbors = ast.literal_eval(contiguity_list[contiguity_list['Precinct']==id]['Neighbors'].values.tolist()[0])
        # needed to convert string to list because the csv encodes the list as a string
        for neighbor in neighbors:
            if neighbor in district_nodes:
                district_graph.add_edge(id,neighbor)
    if(print_isolates):
        print(list(nx.isolates(district_graph)))
    return nx.is_connected(district_graph)

In [162]:
def getDistrictPopulations(assignment,data_file, num_district):
    population = {}
    for i in range (1,num_district+1):
        population[i] = data_file[data_file['GEOID20'].isin(assignment[assignment['District']==i]['GEOID20'])]['Total_2020_Total'].sum()
    return population

In [163]:
def getDistrictShape(district_id, assignment, boundaries):
    list_precincts = assignment[assignment['District']==district_id]['GEOID20']
    precinct_shapes = []
    for i in list_precincts:
        if shape(boundaries[i]).geom_type == 'Polygon':
            precinct_shapes.append(Polygon(shape(boundaries[i])))
        elif shape(boundaries[i]).geom_type == 'MultiPolygon':
            precinct_shapes.append(MultiPolygon(shape(boundaries[i])))      
    district_shape = shapely.ops.unary_union(precinct_shapes)
    #print(district_shape)
    return district_shape

In [164]:
def pp_compactness(geom): # Polsby-Popper
    p = geom.length
    a = geom.area    
    return (4*np.pi*a)/(p*p)

def box_reock_compactness(geom): # Reock on a rectangle bounding box
    a = geom.area 
    bb = geom.bounds # bounds gives you the minimum bounding box (rectangle)
    bba = abs(bb[0]-bb[2])*abs(bb[1]-bb[3])
    return a/bba

# This Notebook will help you get started on NJ
The data is in Canvas, you should upload it to your Google Drive first (if using Colab), or local filesystem (if using Jupyter).

### This is the current assignment of precinct to congressional districts (12 of them for NJ)
Note that the map shown in DRA is slightly different. This is because some precincts are split in the real assignment, and some additional precinct are created to handle special situations such as prisoners and overseas citizens. You can ignore this for the class project and just use the data and functions provided.

In [165]:
nj_current_assignment = pd.read_csv('Map_Data/precinct-assignments-congress-nj.csv')
nj_current_assignment

,GEOID20,District
0,34033020001,2
1,34033001501,2
2,34033042009,2
3,34033000703,2
4,34033042001,2
...,...,...
6356,34039045302,7
6357,34039045303,7
6358,34039045404,7
6359,34039045801,7


### This is the current demographic and voter data
The data has a lot of attributes that lists voters of different demographics and parties in different elections. You can look at the data Dictionary on Canvas to get details. For this recitation we will only keep votes from the 2020  presidential election and the total 2020 population counts. You can use additional columns (e.g., Governor's elections results, voting age (VAP) population counts, or the composite Dem/Rep score)

In [166]:
nj_precinct_data = pd.read_csv('Map_Data/precinct-data-congress-nj.csv')
keepcolumns = ['GEOID20','District','Total_2020_Pres','Dem_2020_Pres','Rep_2020_Pres','Total_2020_Total','White_2020_Total','Hispanic_2020_Total','Black_2020_Total','Asian_2020_Total','Native_2020_Total','Pacific_2020_Total']
nj_precinct_data = nj_precinct_data[keepcolumns]
nj_precinct_data

,GEOID20,District,Total_2020_Pres,Dem_2020_Pres,Rep_2020_Pres,Total_2020_Total,White_2020_Total,Hispanic_2020_Total,Black_2020_Total,Asian_2020_Total,Native_2020_Total,Pacific_2020_Total
0,34001005101,2,876,393,472,1240,946,128,102,66,24,0
1,34001005102,2,852,450,388,1913,1331,211,286,84,38,4
2,34001005103,2,1206,517,672,1760,1375,177,78,106,20,2
3,34001005201,2,828,348,469,1311,906,168,150,64,50,5
4,34001005202,2,868,579,282,1892,537,336,598,450,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6356,34041115002,7,606,182,418,737,714,11,3,8,0,0
6357,34041115003,7,617,187,418,934,820,60,26,10,14,0
6358,34041115004,7,478,160,308,697,602,66,16,4,5,0
6359,34041115005,7,592,201,381,930,831,47,27,11,12,0


### This is the precinct boundary data (uses shapely)

This is data that represents the geography of the districts. It is needed to test for contiguity, or for any districting partitioning method based on geography. The data is in Shapely format. Each district is represented as a set of points that are connected to create the district shape (in the long/lat coordinates). Shapely geometric functions can be used to compare the shapes. These can be quite inefficient to run, so I am also providing you a pre-computed index that, for each district, lists the districts that are contiguous to it. You can see the code to generate the index in Contiguity.ipynb.

To manipulate the shapes, cast them into Shapely Polygons (see example below) and you can use the Polygon properties and functions: https://shapely.readthedocs.io/en/stable/reference/shapely.Polygon.html#shapely.Polygon

In [167]:
shpfile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.shp'
dbffile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.dbf'
shxfile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.shx'


shpfile = shp.Reader(shp=shpfile, shx=shxfile, dbf=dbffile)
nj_precinct_boundaries={}
for sr in shpfile.iterShapeRecords():
    geom = sr.shape # get geo bit
    rec = sr.record # get db fields
    nj_precinct_boundaries[rec[3]]=geom

### This is the precinct boundary data 

This use the contiguity index I have pre-computed using Contiguity.ipynb, that is stored in Contiguity_nj.csv. 

In [168]:
nj_contiguity = pd.read_csv('Contiguity_nj.csv', header=None)

In [169]:
for i in range(1,13):
    print("District "+str(i)+" "+str(isDistrictContiguous(i, nj_current_assignment, nj_contiguity)))

District 1 True
District 2 True
District 3 True
District 4 True
District 5 True
District 6 True
District 7 True
District 8 True
District 9 True
District 10 True
District 11 True
District 12 True


In [170]:
#Compactness of the current assignment
for district in range(1,13):
    print("D"+str(district)+" PP : "+str(pp_compactness(getDistrictShape(district,nj_current_assignment,nj_precinct_boundaries))))
    print("D"+str(district)+" BR : "+str(box_reock_compactness(getDistrictShape(district,nj_current_assignment,nj_precinct_boundaries))))
    

D1 PP : 0.41768102211569347
D1 BR : 0.45075813446417157
D2 PP : 0.2632665176502347
D2 BR : 0.38278056561756263
D3 PP : 0.2280937682959879
D3 BR : 0.38134920642809134
D4 PP : 0.24812480573284196
D4 BR : 0.5390173747196018
D5 PP : 0.2410116694999733
D5 BR : 0.36320426268176653
D6 PP : 0.14677124653853732
D6 BR : 0.32853496486220907
D7 PP : 0.20246375771704353
D7 BR : 0.44049249082841035
D8 PP : 0.11227347882175574
D8 BR : 0.36670629634952656
D9 PP : 0.1683197710884751
D9 BR : 0.29705227593212374
D10 PP : 0.12061263370064262
D10 BR : 0.34528827672774703
D11 PP : 0.22236600778446886
D11 BR : 0.5557086439792166
D12 PP : 0.1620092442171186
D12 BR : 0.38520439164401626


In [171]:
# District Population of the current assignment
print(getDistrictPopulations(nj_current_assignment,nj_precinct_data, 12))

{1: np.int64(775340), 2: np.int64(778354), 3: np.int64(778489), 4: np.int64(767834), 5: np.int64(774454), 6: np.int64(778516), 7: np.int64(785173), 8: np.int64(800074), 9: np.int64(766863), 10: np.int64(746178), 11: np.int64(769523), 12: np.int64(768196)}


# A simple geographical  redistricting strategy

We can create a simple geopgraphical map, like we did for NH. In this case, we have 12 districts, so let's splitting the district in half North/South, and in 6th  East/West. 
New Hampshire's bounding box is (-75.559614,38.928519,-73.893979,41.357423) (https://anthonylouisdagostino.com/bounding-boxes-for-all-us-states/)
So let's start by splitting the state approximately though the middle longitude (-74.72) : everything west of longitude -71.583934 is in odd Districts, everything east is in even Districts. We will use the precinct centroids to assign them. Then we will divide each half per latitude on the ranges  (38.92, 39.3, 39.7, 40.1, 40.5,40.9,41.35)
Import the Map to DRA to look at it.

In [172]:
nj_longlat_assignment = nj_current_assignment.copy()
nj_longlat_assignment['District'] = 0
for index, row in nj_longlat_assignment.iterrows():
    try:
        if shape(nj_precinct_boundaries[row['GEOID20']]).geom_type == 'Polygon':
            centroid = Polygon(shape(nj_precinct_boundaries[row['GEOID20']])).centroid
        elif shape(nj_precinct_boundaries[row['GEOID20']]).geom_type == 'MultiPolygon':
            centroid = MultiPolygon(shape(nj_precinct_boundaries[row['GEOID20']])).centroid
        else:
            print(shape(nj_precinct_boundaries[row['GEOID20']]).geom_type)
            pass
        if centroid.x <= -74.72:
            if centroid.y <= 39.3:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 1
            elif centroid.y <= 39.7:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 3
            elif centroid.y <= 40.1:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 5
            elif centroid.y <= 40.5:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 7
            elif centroid.y <= 40.9:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 9
            else:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 11
        else:
            if centroid.y <= 39.3:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 2
            elif centroid.y <= 39.7:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 4
            elif centroid.y <= 40.1:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 6
            elif centroid.y <= 40.5:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 8
            elif centroid.y <= 40.9:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 10
            else:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 12
    except KeyError: 
        pass
#print(nh_longitude_assignment)
nj_longlat_assignment.to_csv('Recitation maps/nj_longlat_map.csv',index=False)

In [173]:
#Compactness of this Longlat assignment
for district in range(1,13):
    print("D"+str(district)+" PP : "+str(pp_compactness(getDistrictShape(district,nj_longlat_assignment,nj_precinct_boundaries))))
    print("D"+str(district)+" BR : "+str(box_reock_compactness(getDistrictShape(district,nj_longlat_assignment,nj_precinct_boundaries))))
    

D1 PP : 0.24444322502174795
D1 BR : 0.39921574154934536
D2 PP : 0.21732375139633123
D2 BR : 0.3033805826444085
D3 PP : 0.3012927570664991
D3 BR : 0.7281613087250183
D4 PP : 0.3019375759906228
D4 BR : 0.6388831361756292
D5 PP : 0.29897783517755583
D5 BR : 0.5136294328843819
D6 PP : 0.22596255307162377
D6 BR : 0.5362987383285662
D7 PP : 0.21866681223528045
D7 BR : 0.4337413755961903
D8 PP : 0.3513739185552307
D8 BR : 0.82797734690796
D9 PP : 0.37261194943216025
D9 BR : 0.7269867982048943
D10 PP : 0.28764401412152446
D10 BR : 0.6900841778845721
D11 PP : 0.34638282192061104
D11 BR : 0.47137706735305646
D12 PP : 0.318631303145351
D12 BR : 0.5828294919378438


In [174]:
# District Population of this longlat assignment
print(getDistrictPopulations(nj_longlat_assignment,nj_precinct_data, 12))


{1: np.int64(80406), 2: np.int64(22492), 3: np.int64(317218), 4: np.int64(274411), 5: np.int64(1124427), 6: np.int64(563593), 7: np.int64(244052), 8: np.int64(1471840), 9: np.int64(230117), 10: np.int64(3848094), 11: np.int64(61026), 12: np.int64(1051318)}


# Now create your own redistricting maps
Remember to check for contiguity, and to ensure that the population of the districts are balanced (which is not the case in the example above.)

## Fair Map Seeding Algorithm

This section implements the seeding algorithm for the fair map. The goal is to select 12 initial seed precincts (one per district) that are:
- Geographically spread out across the state
- Have population sizes initially close to the average precinct population
- Consider demographic trends and try to pair demographic groups 
- Not biased towards any political party 
- Located in areas that allow for compact district growth

- Since final population of each district should be around NJPopulation/12.
- The seed population of each district should be 10% of NJ Population/12


In [175]:
def get_precinct_centroid(geoid, boundaries):
    """Get the centroid coordinates for a single precinct."""
    try:
        if shape(boundaries[geoid]).geom_type == 'Polygon':
            centroid = Polygon(shape(boundaries[geoid])).centroid
        elif shape(boundaries[geoid]).geom_type == 'MultiPolygon':
            centroid = MultiPolygon(shape(boundaries[geoid])).centroid
        else:
            return None
        return (centroid.x, centroid.y)  # (longitude, latitude)
    except (KeyError, Exception):
        return None

def get_district_centroid(district_id, assignment, boundaries):
    """Get the centroid coordinates for a district (all precincts in that district)."""
    district_precincts = assignment[assignment['District'] == district_id]['GEOID20'].tolist()
    precinct_centroids = []
    for geoid in district_precincts:
        centroid = get_precinct_centroid(geoid, boundaries)
        if centroid:
            precinct_centroids.append(centroid)
    
    if not precinct_centroids:
        return None
    
    # Calculate average centroid (simple approach)
    avg_lon = sum(c[0] for c in precinct_centroids) / len(precinct_centroids)
    avg_lat = sum(c[1] for c in precinct_centroids) / len(precinct_centroids)
    return (avg_lon, avg_lat)

def get_precinct_neighbors(geoid, contiguity_df):
    """Get the list of neighboring precincts for a given precinct."""
    df = contiguity_df.copy()
    df.columns = ['Precinct', 'Neighbors']
    try:
        neighbors_str = df[df['Precinct'] == geoid]['Neighbors'].values[0]
        return ast.literal_eval(neighbors_str)
    except (KeyError, IndexError, ValueError):
        return []


total_nj_population = nj_precinct_data['Total_2020_Total'].sum()
avg_district_population = total_nj_population / 12
advisedSeedPopulation = avg_district_population * 0.10


def select_population_balanced_seeds(precinct_data, boundaries, contiguity_df, num_districts=12):
    import pandas as pd
    import math
    
    # Build coordinates for every precinct from shapefile centroids
    coords = {}
    for geoid in precinct_data['GEOID20']:
        centroid = get_precinct_centroid(geoid, boundaries)
        if centroid is not None:
            coords[geoid] = centroid  # (lon, lat)

    # Filter precinct_data to only those with valid centroids
    precinct_data = precinct_data[precinct_data['GEOID20'].isin(coords.keys())].copy()
    
    # Prepare candidate seeds with basic filtering like before
    avg_precint_pop = precinct_data['Total_2020_Total'].mean()
    pop_low = avg_precint_pop * 0.5
    pop_high = avg_precint_pop * 1.5

    def is_good_seed_candidate(row):
        geoid = row['GEOID20']
        pop = row['Total_2020_Total']
        if pop <= 0:
            return False
        neighbors = get_precinct_neighbors(geoid, contiguity_df)
        if len(neighbors) == 0:
            return False
        if not (pop_low <= pop <= pop_high):
            return False
        return True
    
    candidates = precinct_data[precinct_data.apply(is_good_seed_candidate, axis=1)].copy()
    if candidates.empty:
        # fallback to any with neighbors
        candidates = precinct_data[precinct_data['GEOID20'].map(
            lambda g: len(get_precinct_neighbors(g, contiguity_df)) > 0
        )].copy()

    # Add lon/lat columns
    candidates['lon'] = candidates['GEOID20'].map(lambda g: coords[g][0])
    candidates['lat'] = candidates['GEOID20'].map(lambda g: coords[g][1])

    # Calculate center of candidates for initial seed pick
    mean_lon = candidates['lon'].mean()
    mean_lat = candidates['lat'].mean()

    def euclidean(p1, p2):
        return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

    seeds = []
    assigned_pop = 0
    total_pop = candidates['Total_2020_Total'].sum()
    ideal_seed_pop = total_pop / num_districts

    # Select the first seed: closest to center
    candidates['dist_to_center'] = ((candidates['lon'] - mean_lon)**2 + 
                                    (candidates['lat'] - mean_lat)**2).pow(0.5)
    first_seed = candidates.sort_values('dist_to_center').iloc[0]
    seeds.append(first_seed['GEOID20'])
    assigned_pop += first_seed['Total_2020_Total']
    candidates = candidates[candidates['GEOID20'] != first_seed['GEOID20']]

    # Now select remaining seeds balancing distance and population:
    while len(seeds) < num_districts:
        # Compute for each candidate:
        # - min distance to existing seeds (to spread geographically)
        # - difference of candidate pop to ideal_seed_pop (to balance pop)

        weight_pop = 0.9
        weight_dist = 1 - weight_pop
        max_dist = candidates['lon'].max() - candidates['lon'].min()  # rough max lon range as proxy for max distance
        max_pop_diff = candidates['Total_2020_Total'].max() - candidates['Total_2020_Total'].min()
        def score_candidate(row):
            dist_to_seed = min(euclidean(coords[row['GEOID20']], coords[s]) for s in seeds)
            pop_diff = abs(row['Total_2020_Total'] - ideal_seed_pop)

            dist_norm = dist_to_seed / max_dist if max_dist > 0 else 0
            pop_norm = pop_diff / max_pop_diff if max_pop_diff > 0 else 0

            return weight_dist * dist_norm - weight_pop * pop_norm


        candidates['score'] = candidates.apply(score_candidate, axis=1)
        next_seed = candidates.sort_values('score', ascending=False).iloc[0]

        seeds.append(next_seed['GEOID20'])
        assigned_pop += next_seed['Total_2020_Total']
        candidates = candidates[candidates['GEOID20'] != next_seed['GEOID20']]

    return seeds


In [176]:
import pandas as pd
import math
import random
from collections import deque

def flood_fill_population_first(
        initial_assignment,        # DataFrame with at least 'GEOID20' and 'District' columns; unassigned should be -1 or 0
        precinct_data,             # precinct dataframe with 'GEOID20' and 'Total_2020_Total'
        boundaries,                # boundaries mapping used by get_precinct_centroid / get_district_centroid
        contiguity_df,             # contiguity df same format used by get_precinct_neighbors / isDistrictContiguous
        seeds=None,                # list of GEOID20 seeds (length == num_districts). If None, call select_population_balanced_seeds.
        num_districts=12,
        pop_tolerance=0.05,        # how much over ideal population we allow when accepting a precinct (fraction)
        max_iterations=2000,     # safety cap
        verbose=True
    ):
    """
    Population-first flood fill assignment.
    Returns a copy of assignment DataFrame with 'District' filled (1..num_districts).
    """

    # --- PREP ---
    assignment = initial_assignment.copy()
    # normalize unassigned flag
    assignment['District'] = assignment['District'].fillna(-1).astype(int)
    unassigned_mask = ~assignment['GEOID20'].isin(precinct_data['GEOID20'])
    if unassigned_mask.any():
        # remove precincts that aren't in precinct_data
        assignment = assignment[~unassigned_mask].copy()

    total_population = precinct_data['Total_2020_Total'].sum()
    ideal_pop = total_population / float(num_districts)
    max_allowed_pop = ideal_pop * (1.0 + pop_tolerance)

    # compute seeds if not provided
    if seeds is None:
        seeds = select_population_balanced_seeds(precinct_data, boundaries, contiguity_df, num_districts=num_districts)

    if len(seeds) != num_districts:
        raise ValueError("Seeds length != num_districts (got {}).".format(len(seeds)))

    # initialize
    visited = set()
    next_precincts = {i: deque() for i in range(1, num_districts + 1)}
    district_pops = {i: 0 for i in range(1, num_districts + 1)}
    district_sizes = {i: 0 for i in range(1, num_districts + 1)}

    # assign seeds
    for idx, seed_geoid in enumerate(seeds, start=1):
        # assign seed to district idx
        if seed_geoid not in assignment['GEOID20'].values:
            # seed not in assignment/precinct_data (rare); choose the closest candidate instead
            if verbose:
                print(f"Warning: seed {seed_geoid} not present in assignment. Picking random candidate for district {idx}.")
            candidate = precinct_data.sample(1)['GEOID20'].values[0]
            seed_geoid = candidate

        assignment.loc[assignment['GEOID20'] == seed_geoid, 'District'] = idx
        visited.add(seed_geoid)
        pop = int(precinct_data.loc[precinct_data['GEOID20'] == seed_geoid, 'Total_2020_Total'].iloc[0])

        district_pops[idx] += pop
        district_sizes[idx] += 1

        # seed neighbors into queue
        try:
            neighbors = get_precinct_neighbors(seed_geoid, contiguity_df)
        except Exception:
            neighbors = []
        for n in neighbors:
            if n not in visited:
                next_precincts[idx].append(n)

    # helper to get current smallest-pop district that still can grow
    def smallest_district():
        # returns district id with smallest pop, tie-break randomly
        items = sorted(district_pops.items(), key=lambda kv: (kv[1], random.random()))
        return items[0][0]

    # map for quick pop lookup
    pop_lookup = precinct_data.set_index('GEOID20')['Total_2020_Total'].to_dict()

    precincts_added = sum(district_sizes.values())
    total_precincts = assignment.shape[0]
    iterations = 0

    # main growth loop
    while precincts_added < total_precincts and iterations < max_iterations:
        iterations += 1

        i = smallest_district()

        # if this district has no frontier, try to seed it with nearest unvisited precinct (heuristic)
        if not next_precincts[i]:
            # try to add any neighbor-of-assigned precinct that borders this district
            added = False
            # iterate assigned precincts in this district and push unvisited neighbors
            assigned_geoids = assignment[assignment['District'] == i]['GEOID20'].tolist()
            for g in assigned_geoids:
                nb = get_precinct_neighbors(g, contiguity_df)
                for n in nb:
                    if n not in visited and n not in next_precincts[i]:
                        next_precincts[i].append(n)
                        added = True
            if not added:
                # last-ditch: find any unvisited precinct and append
                remaining = [g for g in assignment[assignment['District'] <= 0]['GEOID20'].tolist() if g not in visited]
                if remaining:
                    next_precincts[i].append(remaining[0])

        # If still empty skip
        if not next_precincts[i]:
            # nothing we can do this iteration
            if iterations % 10000 == 0 and verbose:
                print("Iteration {}: no frontier for district {}, continuing...".format(iterations, i))
            continue

        current_precinct = next_precincts[i].popleft()
        # Skip if already assigned/visited (may have been assigned by other district)
        if current_precinct in visited or assignment.loc[assignment['GEOID20'] == current_precinct, 'District'].values[0] > 0:
            continue

        # Tentatively assign precinct to district i
        assignment.loc[assignment['GEOID20'] == current_precinct, 'District'] = i
        visited.add(current_precinct)
        precinct_pop = int(pop_lookup.get(current_precinct, 0))
        district_pops[i] += precinct_pop
        district_sizes[i] += 1
        precincts_added += 1

        # Contiguity & population checks:
        is_contig = True
        try:
            is_contig = isDistrictContiguous(i, assignment, contiguity_df, print_isolates=False)
        except Exception:
            # If contiguity check errors, be conservative and treat as not contiguous
            is_contig = False

        is_pop_ok = district_pops[i] <= max_allowed_pop or district_pops[i] <= ideal_pop  # allow up to tolerance over ideal

        if not (is_contig and is_pop_ok):
            # revert assignment
            if verbose and iterations % 500 == 0:
                print(f"Reverting {current_precinct} from district {i} (contig={is_contig}, pop={district_pops[i]} > {max_allowed_pop})")
            assignment.loc[assignment['GEOID20'] == current_precinct, 'District'] = -1
            visited.remove(current_precinct)
            district_pops[i] -= precinct_pop
            district_sizes[i] -= 1
            precincts_added -= 1
            # do not add neighbors for this precinct; it may be re-enqueued later

            # optionally, we can push this precinct to a lower-priority hold queue to try later
            continue

        # If assignment accepted, enqueue this precinct's unvisited neighbors
        neighbors = get_precinct_neighbors(current_precinct, contiguity_df)
        for nb in neighbors:
            if nb not in visited and nb not in next_precincts[i]:
                next_precincts[i].append(nb)

        # periodic progress print
        if precincts_added % 100 == 0 and verbose:
            print(f"Added {precincts_added}/{total_precincts} precincts after {iterations} iterations")
            # optionally show small summary
            if verbose:
                pops = getDistrictPopulations(assignment, precinct_data, num_districts)
                print("district pops (sample):", {k: pops[k] for k in sorted(pops)[:6]})

    if iterations >= max_iterations and verbose:
        print("Reached max_iterations. Some precincts may remain unassigned.")

    # If there are any unassigned precincts left, assign them greedily to nearest district centroid
    unassigned_df = assignment[assignment['District'] <= 0].copy()
    if not unassigned_df.empty:
        if verbose:
            print(f"Filling {len(unassigned_df)} remaining precincts by nearest district centroid.")
        # compute centroids for each district
        district_centroids = {}
        for d in range(1, num_districts + 1):
            try:
                c = get_district_centroid(d, assignment, boundaries)
            except Exception:
                c = None
            if c is None:
                # fallback: choose centroid of seed
                seed_geoid = seeds[d-1]
                c = get_precinct_centroid(seed_geoid, boundaries)
            district_centroids[d] = c

        # assign each unassigned to nearest centroid (and try contiguity)
        for _, row in unassigned_df.iterrows():
            g = row['GEOID20']
            centroid_g = get_precinct_centroid(g, boundaries)
            if centroid_g is None:
                # assign randomly
                best_d = random.randint(1, num_districts)
            else:
                # find nearest district centroid
                best_d = min(district_centroids.keys(),
                             key=lambda d: math.hypot(centroid_g[0] - (district_centroids[d][0] if district_centroids[d] else 0),
                                                      centroid_g[1] - (district_centroids[d][1] if district_centroids[d] else 0)))
            assignment.loc[assignment['GEOID20'] == g, 'District'] = best_d
            # no contiguity reversion here (final pass)

    # final diagnostics
    if verbose:
        pops = getDistrictPopulations(assignment, precinct_data, num_districts)
        print("Final district populations:", pops)
        print("Total precincts assigned:", assignment[assignment['District'] > 0].shape[0])

    return assignment


In [177]:
def fix_all_uncontiguous_precincts(
    assignment,
    contiguity_df,
    precinct_data,
    district_ids=None,
    population_col="Total_2020_Total"
):
    """Reassign isolated precincts to neighboring districts with the lowest population."""
    assignment = assignment.copy()
    contiguity_df = contiguity_df.copy()

    # normalize contiguity columns
    contiguity_df.columns = ["Precinct", "Neighbors"]
    neighbor_map = {}
    for geoid, nb in zip(contiguity_df["Precinct"], contiguity_df["Neighbors"]):
        try:
            neighbor_map[geoid] = ast.literal_eval(nb)
        except Exception:
            neighbor_map[geoid] = []

    # build population lookup and current assignment map
    pop_map = dict(zip(precinct_data["GEOID20"], precinct_data[population_col]))
    assignment_map = dict(zip(assignment["GEOID20"], assignment["District"]))

    if district_ids is None:
        district_ids = sorted(d for d in set(assignment_map.values()) if d > 0)

    district_pop = {d: 0 for d in district_ids}
    for geoid, dist in assignment_map.items():
        if dist in district_pop:
            district_pop[dist] += pop_map.get(geoid, 0)

    changed = True
    while changed:
        changed = False
        for geoid, district in assignment_map.items():
            if district <= 0:
                continue
            neighbors = neighbor_map.get(geoid, [])
            same_district_neighbors = [n for n in neighbors if assignment_map.get(n) == district]
            if same_district_neighbors:
                continue

            neighbor_districts = {assignment_map.get(n) for n in neighbors if assignment_map.get(n, -1) > 0}
            if not neighbor_districts:
                continue

            best_district = min(neighbor_districts, key=lambda nd: district_pop.get(nd, float("inf")))
            if best_district == district:
                continue

            pop = pop_map.get(geoid, 0)
            district_pop[district] -= pop
            district_pop[best_district] = district_pop.get(best_district, 0) + pop
            assignment_map[geoid] = best_district
            assignment.loc[assignment['GEOID20'] == geoid, 'District'] = best_district
            changed = True

    return assignment


In [178]:
# assume you already loaded nj_precinct_data, nj_current_assignment, nj_contiguity, boundaries
seeds = select_population_balanced_seeds(
    nj_precinct_data,
    nj_precinct_boundaries,
    nj_contiguity,
    num_districts=12
)
result = flood_fill_population_first(
    nj_current_assignment,
    nj_precinct_data,
    nj_precinct_boundaries,
    nj_contiguity,
    seeds=seeds,
    num_districts=12
)
fixed_assignment = fix_all_uncontiguous_precincts(
    result,
    nj_contiguity,
    nj_precinct_data
)
fixed_assignment.to_csv('ff_population_first_assignment.csv', index=False)


Reached max_iterations. Some precincts may remain unassigned.
Final district populations: {1: np.int64(774361), 2: np.int64(776193), 3: np.int64(780679), 4: np.int64(767835), 5: np.int64(774448), 6: np.int64(776345), 7: np.int64(787347), 8: np.int64(800074), 9: np.int64(769024), 10: np.int64(746161), 11: np.int64(769522), 12: np.int64(767005)}
Total precincts assigned: 6361
